In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from natsort import natsorted
from os.path import join as pjoin
from scipy.stats import zscore
from dask.distributed import Client, LocalCluster

sys.path.append('../../')
import circletrack_neural as ctn

project_dir = 'MultiCon_Imaging'
experiment_dir = 'MultiCon_Imaging6'
mouse_list = ['mc55']
dpath = f'../../../{project_dir}/{experiment_dir}/output/aligned_minian/'
spath = f'../../../{project_dir}/{experiment_dir}/output/reward_modulation/'
data_of_interest = 'aligned_minian'
data_type = 'C'
starting_idx = 13
window_size = 90 ## in frames
sampling_rate = 1/30
num_simulations = 500
percentiles = (5, 95)
correct_size = (window_size * 2) + 1 ## since inclusive on both ends

xr.set_options(keep_attrs=True)

## Set seed
np.random.seed(24601)

## Loop through mouse list
for mouse in mouse_list:
    mpath = pjoin(dpath, f'{mouse}/{data_type}')
    for idx, session in enumerate(natsorted(os.listdir(mpath))):
        if idx < starting_idx:
            pass 
        else:
            print(session)
            ## Create cluster
            cluster = LocalCluster(n_workers=14,
                                memory_limit='8GB',
                                resources={'MEM': 1},
                                threads_per_worker=2,
                                dashboard_address=':8787')
            client = Client(cluster)

            save_path = pjoin(spath, f'{mouse}/{data_type}')
            C = xr.open_dataset(pjoin(mpath, session))[data_type]
            num_neurons = C.shape[0]
            ## Normalize data
            zdata = xr.apply_ufunc(
                    zscore,
                    C.chunk({'frame': -1, 'unit_id': 50}),
                    input_core_dims=[['frame']],
                    output_core_dims=[['frame']],
                    kwargs={'axis': 1},
                    dask='parallelized'
            ).compute()
            ## Compute actual reward-related activity
            windowed_data, windowed_sem, rw_pre, rw_post, rw_one_array, rw_two_array = ctn.reward_activity(zdata, 
                                                                                                            correct_size=correct_size, 
                                                                                                            window_size=window_size,
                                                                                                            reward_one=zdata.attrs['reward_one'],
                                                                                                            reward_two=zdata.attrs['reward_two'])
            ## Compute shuffle distribution
            df = ctn.compute_shuffled_rw_diff(zdata, 
                                            correct_size=correct_size, 
                                            window_size=window_size, 
                                            num_simulations=num_simulations)
            ## Get difference in observed activity around reward locations
            ## and compare observed activity to a shuffle distribution
            rw_one_diff = rw_post[:, 0] - rw_pre[:, 0]
            rw_two_diff = rw_post[:, 1] - rw_pre[:, 1]
            rw_one_pos = np.array([rw_one_diff[neuron] > np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], percentiles[1]) for neuron in np.arange(0, num_neurons)])
            rw_two_pos = np.array([rw_two_diff[neuron] > np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], percentiles[1]) for neuron in np.arange(0, num_neurons)])
            rw_one_neg = np.array([rw_one_diff[neuron] < np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], percentiles[0]) for neuron in np.arange(0, num_neurons)])
            rw_two_neg = np.array([rw_two_diff[neuron] < np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], percentiles[0]) for neuron in np.arange(0, num_neurons)])
            ## Assign boolean arrays saying whether that unit_id is pos/neg modulated by reward
            C = C.assign_coords(rw_one_pos=('unit_id', rw_one_pos),
                                rw_two_pos=('unit_id', rw_two_pos),
                                rw_one_neg=('unit_id', rw_one_neg),
                                rw_two_neg=('unit_id', rw_two_neg))
            ## Save data
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            C.to_netcdf(pjoin(save_path, f'{session}'))

            ## Close cluster     
            client.close()
            cluster.close()

mc55_C_1.nc


c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\distributed\client.py:3370: UserWarning: Sending large graph of size 23.61 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


mc55_C_2.nc


2026-01-12 09:47:44,136 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bokeh.protocol.exceptions.ProtocolError: Token is expired. Configure the app with a larger value for --session-token-expiration if necessary
2026-01-12 09

mc55_C_4.nc


...

2026-01-13 14:36:46,508 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bokeh.protocol.exceptions.ProtocolError: Token is expired. Configure the app with a larger value for --session-token-expiration if necessary
2026-01-13 14

mc55_C_14.nc


2026-01-13 18:40:05,911 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bokeh.protocol.exceptions.ProtocolError: Token is expired. Configure the app with a larger value for --session-token-expiration if necessary
2026-01-13 18

mc55_C_15.nc


2026-01-13 23:17:34,719 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\tornado\web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
  File "c:\Users\ambag\miniconda3\envs\calcium\Lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bokeh.protocol.exceptions.ProtocolError: Token is expired. Configure the app with a larger value for --session-token-expiration if necessary
2026-01-13 23

In [ ]:
client.close()
cluster.close()